## Initialize the Agent

In [ ]:
import os

from llama_stack_client import Agent, AgentEventLogger
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types.toolgroup_register_params import McpEndpoint


model = "llama3.2:3b-instruct-fp16"
os.environ["INFERENCE_MODEL"] = model
client = LlamaStackAsLibraryClient('ollama')
client.initialize()


client.toolgroups.register(
    toolgroup_id="mcp::custom",
    provider_id="model-context-protocol",
    mcp_endpoint=McpEndpoint(uri="http://127.0.0.1:5000/sse"),
)

instructions="""
You are a smart assistant that helps users with his questions.
You have differents available tools and you must use to retrieve information or perform actions.
You must not generate code or scripts.
Use the tool to interact with differents system.
No simulate the output, perform all operations with tools available.
You are not allowed to infer or hallucinate any answer.
You must use the kube_command tool to execute all kubectl commands.
If the command fails, then show the error and stop the execution.

You must interact with the Kubernetes cluster **only** via the `kube_command` tool. Never fabricate results. Every action must be performed by calling the tool.

When applying manifests, never use heredocs (`<<EOF`). Instead, use the following pattern:

echo "<full YAML manifest>" | kubectl apply -f -

Avoid summarizing or simulating command outputs. Only describe what you're doing **before** issuing the tool call. Let the tool return the actual result.

Output only valid `kube_command(cmd=...)` calls, nothing else.

The available tools are:
- kube_command(cmd)
// You have to explicity add the entire command with kubectl
// example: kubectl get pods
"""

optimized_instructions = """
You are a smart assistant that helps users with his questions.
You have differents available tools and you must use to retrieve information or perform actions.
You must not generate code or scripts.
No simulate the output, perform all operations with tools available.
You are force to use kube_command tool to create resources in kubernetes, so if the user ask create something you MUST use the tool.
Not use yaml to create resources, Just use kubectl commands
You have to add kubectl in the commands you pass to kubec_command tool, for instance kubectl get pods 
"""

agent = Agent(
  client,
  model=model,
  instructions=optimized_instructions,
  tools=["mcp::custom"],
  sampling_params={
    "strategy": {"type": "top_p", "temperature": 0.2, "top_p": 0.9},
  },
)
session_id = agent.create_session('new_session')

## Executing a task with explicit instructions

In [ ]:


prompt = "Use the kube_command to execute `kubectl get pods -A`. Show me the output"
print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

## Executing a simple task leaves it to the LLM’s interpretation

In [ ]:
prompt = "Create a new namespace called mcp-tests."
print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

## Executing multiples tasks

In [ ]:
prompt = "Deploy the hashicorp/http-echo:latest service in mcp-tests namespace. It should be accessible from outside cluster. Then use the yaml created and call the kube_cmd tool to create the resources in the cluster. I don't want the yaml I want you use the kube_cmd tool to create the resources. You are force to call the tool"

print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()